In [12]:
import numpy as np
import pandas as pd
import seaborn as sns

In [13]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [14]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [15]:
mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset()
df_all, df_census = utils.merge_dataset(df_train, df_test, pop=False, unemploy=False, census=False, coord=True, fix_pop=True, add_location=False, outlier=False)

/tmp/working/code-analysis/../modules/utils.py:222: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  if fix_pop:


In [7]:
# df_submission = pd.read_csv('../submission/test_subm_accum_2023-02-25_21:02:46.csv')

In [17]:
df_all = preprocess.add_lag_features(df_all)

add lag features: max_scale=40


In [21]:
trend_dict = preprocess.get_trend_multi(df_all)

In [22]:
trend_dict2 = preprocess.get_trend_multi(df_all, origin=True)